In [1]:
from __future__ import division
from __future__ import print_function
import caffe
import numpy as np
import os
import sys
from datetime import datetime
from PIL import Image
from IPython.display import Image
from IPython.display import clear_output
import time

In [2]:
import caffe

In [3]:
def log_progress(sequence, title='', every=1, size=None, end_msg='', removeAtTheEnd=False):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    if isinstance(sequence, dict):
                        val = ''
                    else:
                        val=sequence[index-1]
                            
                    label.value = u'{title} : {value} ({index} / {size})'.format(
                        title=title,
                        value=val,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = end_msg
        if removeAtTheEnd:
            box.close()
    

In [4]:
def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

In [5]:
def evaluate(results):
    
    hist_cumul = 0*results[0]
    for hist in results:
        hist_cumul+=hist
    result = {}
    

    acc = np.diag(hist_cumul).sum() / hist_cumul.sum()
    # per-class accuracy
    pc_acc = np.diag(hist_cumul) / hist_cumul.sum(1)
    mean_accuracy= np.nanmean(pc_acc)
    # per-class IU
    iu = np.diag(hist_cumul) / (hist_cumul.sum(1) + hist_cumul.sum(0) - np.diag(hist_cumul))
    mean_iu= np.nanmean(iu)
    freq = hist_cumul.sum(1) / hist_cumul.sum()
    freq_weighted_iu=(freq[freq > 0] * iu[freq > 0]).sum()
    
    print("Accuracy :",acc)
    print("per-class accuracy :",mean_accuracy)
    print("per-class IU :",mean_iu)
    print("per-class IU (frequency weighted) :",freq_weighted_iu)
    
    return acc

In [6]:
weights_path = '/home/jguerry/workspace/dl_data_sunrgbd/MODELS/fcn32s_rgb100_sun/snapshot_iter_105700.caffemodel'
prototxt_path = '/home/jguerry/workspace/dl_data_sunrgbd/MODELS/fcn32s_optim/train_val.prototxt'
caffe_gpu = True
if not caffe_gpu:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0);

In [7]:
#net = caffe.Net(prototxt_path, caffe.TEST)
net = caffe.Net(prototxt_path,weights_path , caffe.TEST)

#### Blobs & Layers

In [8]:
B = [(k, v.data.shape) for k, v in net.blobs.items()]
L = [(k, v[0].data.shape, v[1].data.shape) for k, v in net.params.items() if k != 'upscore37']

In [12]:
L

[('conv1_1', (64, 3, 3, 3), (64,)),
 ('conv1_2', (64, 64, 3, 3), (64,)),
 ('conv2_1', (128, 64, 3, 3), (128,)),
 ('conv2_2', (128, 128, 3, 3), (128,)),
 ('conv3_1', (256, 128, 3, 3), (256,)),
 ('conv3_2', (256, 256, 3, 3), (256,)),
 ('conv3_3', (256, 256, 3, 3), (256,)),
 ('conv4_1', (512, 256, 3, 3), (512,)),
 ('conv4_2', (512, 512, 3, 3), (512,)),
 ('conv4_3', (512, 512, 3, 3), (512,)),
 ('conv5_1', (512, 512, 3, 3), (512,)),
 ('conv5_2', (512, 512, 3, 3), (512,)),
 ('conv5_3', (512, 512, 3, 3), (512,)),
 ('fc6', (4096, 512, 7, 7), (4096,)),
 ('fc7', (4096, 4096, 1, 1), (4096,)),
 ('score_fr37', (37, 4096, 1, 1), (37,))]

In [9]:
results = []
n_cl = net.blobs['score'].channels
hist = np.zeros((n_cl, n_cl))
nb_images_test = 5050
print("###############################################################")
#for idx in log_progress(range(net.layers[0].nb_images), title='iteration', end_msg=str(i),removeAtTheEnd=True):
for idx in log_progress(range(nb_images_test), title='iteration', removeAtTheEnd=True):
    net.forward()
    score_map = net.blobs['score'].data[0].argmax(0)
    gt_map = net.blobs['label'].data[0, 0]
    hist = fast_hist(gt_map.flatten(),score_map.flatten(),n_cl)
    results.append(hist)
evaluate(results)
print("###############################################################")

###############################################################


KeyboardInterrupt: 

In [10]:
evaluate(results)

Accuracy : 0.740929963384
per-class accuracy : 0.479193870023
per-class IU : 0.346634585252
per-class IU (frequency weighted) : 0.610553821466


/data/soft/anaconda2/envs/jg_pfr/lib/python2.7/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in true_divide
/data/soft/anaconda2/envs/jg_pfr/lib/python2.7/site-packages/ipykernel/__main__.py:14: RuntimeWarning: invalid value encountered in true_divide


0.74092996338363248